# Algoritmul REINFORCE (Policy Gradient)

Algoritmul REINFORCE este forma de bază a tehnicilor de *policy gradient*. Scopul său este de a învăța direct o politică stochastică, fără a folosi funcții de valoare precum Q(s, a). Politica este parametrizată și produce probabilități pentru acțiuni.

---

## Obiectiv

Scopul trainingului este maximizarea rewardului total așteptat:

$$
J(\theta) = \mathbb{E}_{\pi_\theta}[R]
$$

Gradientul folosit pentru actualizarea politicii este:

$$
\nabla_\theta J(\theta) = \mathbb{E}\left[ G_t \cdot \nabla_\theta \log \pi_\theta(a_t \mid s_t) \right]
$$

---

## Pașii algoritmului

### 1. Generarea unui episod

Agentul rulează un episod complet și colectează:

- stările 
- acțiunile 
- log-probabilitățile 
- recompensele

Politica este stochastică, deci acțiunile sunt alese prin *sampling* din distribuție.

---

### 2. Calcularea returnurilor

Pentru fiecare pas t calculăm returnul discountat:

$$
G_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \dots
$$

Acesta reprezintă suma rewardurilor începând de la momentul \( t \).

---

### 3. Normalizarea returnurilor

Pentru a reduce varianța gradientului, returnurile pot fi normalizate:

$$
\hat{G_t} = \frac{G_t - \mu}{\sigma}
$$

Acest pas este opțional, dar recomandat pentru stabilitatea antrenării.

---

### 4. Calcularea funcției de pierdere

Funcția de pierdere folosită în REINFORCE este:

$$
L(\theta) = - \sum_t G_t \cdot \log \pi_\theta(a_t \mid s_t)
$$

Interpretare:

- acțiunile care au dus la reward mare primesc gradient pozitiv → devin mai probabile
- acțiunile care au dus la reward mic primesc gradient negativ → devin mai puțin probabile

---

### 5. Actualizarea parametrilor

Parametrii politicii sunt actualizați prin gradient descent:

- se calculează gradientul parametrului theta
- se aplică un optimizer precum Adam

---

## Intuiție

Algoritmul întărește acțiunile care au dus la reward mare și penalizează acțiunile care au dus la reward mic. Este un algoritm simplu și elegant, dar are varianță mare deoarece update-urile se fac doar la finalul episodului.

---

## Limitări

- varianță ridicată a gradientului  
- poate „uita” comportament bun între episoade  
- învățare instabilă în medii complexe  

---

## Observații

REINFORCE este excelent pentru învățare conceptuală și pentru medii simple precum CartPole.  
Algoritmi precum Actor-Critic, A2C sau PPO sunt variante mai stabile construite pe aceleași principii.


In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# ---------------------------
# Policy Network
# ---------------------------
class PolicyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Softmax(dim=-1)   # produce probabilitățile pentru acțiuni
        )

    def forward(self, x):
        return self.fc(x)


# ---------------------------
# REINFORCE Algorithm
# ---------------------------
def compute_returns(rewards, gamma=0.99):
    G = 0
    returns = []
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    return returns


env = gym.make("CartPole-v1")
policy = PolicyNet()
optimizer = optim.Adam(policy.parameters(), lr=0.01)

episodes = 1000
all_rewards = []

for ep in range(episodes):
    state, _ = env.reset()
    log_probs = []
    rewards = []

    done = False
    while not done:
        state_t = torch.tensor(state, dtype=torch.float32)
        probs = policy(state_t)
        dist = torch.distributions.Categorical(probs)

        action = dist.sample()
        log_prob = dist.log_prob(action)

        next_state, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated

        log_probs.append(log_prob)
        rewards.append(reward)
        state = next_state

    # total reward for logging
    total_reward = sum(rewards)
    all_rewards.append(total_reward)

    # compute discounted returns
    returns = compute_returns(rewards)
    returns = torch.tensor(returns, dtype=torch.float32)

    # normalize returns to reduce variance
    returns = (returns - returns.mean()) / (returns.std() + 1e-7)

    # compute loss (REINFORCE)
    loss = 0
    for log_p, G in zip(log_probs, returns):
        loss -= log_p * G

    # gradient step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if ep % 20 == 0:
        print(f"Episode {ep}, reward: {total_reward}")


# ---------------------------
# Plot performance
# ---------------------------
import matplotlib.pyplot as plt

def smooth(data, window=20):
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')


plt.plot(all_rewards, alpha=0.4, label="Raw (noisy)")
plt.plot(smooth(all_rewards, 20), linewidth=2, label="Smoothed")
plt.legend()
plt.grid(True)
plt.title("REINFORCE on CartPole (Smoothed)")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()


### REINFORCE + Baseline

In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# ---------------------------
# Policy Network
# ---------------------------
class PolicyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.fc(x)


# ---------------------------
# Value Network (Baseline)
# ---------------------------
class ValueNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 1)     # produce estimarea V(s)
        )

    def forward(self, x):
        return self.fc(x)


# ---------------------------
# Compute discounted returns
# ---------------------------
def compute_returns(rewards, gamma=0.99):
    G = 0
    returns = []
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    return returns


env = gym.make("CartPole-v1")
policy = PolicyNet()
value_fn = ValueNet()

optimizer = optim.Adam(
    list(policy.parameters()) + list(value_fn.parameters()),
    lr=0.01
)

episodes = 1000
all_rewards = []

for ep in range(episodes):
    state, _ = env.reset()
    log_probs = []
    values = []
    rewards = []

    done = False
    while not done:
        state_t = torch.tensor(state, dtype=torch.float32)

        # policy forward
        probs = policy(state_t)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()

        # critic forward
        value = value_fn(state_t)

        next_state, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated

        log_probs.append(dist.log_prob(action))
        values.append(value)
        rewards.append(reward)

        state = next_state

    total_reward = sum(rewards)
    all_rewards.append(total_reward)

    # discounted returns
    returns = torch.tensor(compute_returns(rewards), dtype=torch.float32)

    # compute advantages
    values = torch.cat(values).squeeze()
    advantages = returns - values.detach()  # IMPORTANT: no grad through values here

    # ---------------------------
    # Loss = Policy loss + Value loss
    # ---------------------------
    policy_loss = -(log_probs[0] * advantages[0])
    for log_p, adv in zip(log_probs, advantages):
        policy_loss += -(log_p * adv)

    # MSE loss pentru baseline
    value_loss = nn.functional.mse_loss(values, returns)

    loss = policy_loss + value_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if ep % 20 == 0:
        print(f"Episode {ep}, reward: {total_reward}")


# ---------------------------
# Plot performance
# ---------------------------
import matplotlib.pyplot as plt

def smooth(data, window=20):
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')

plt.plot(all_rewards, alpha=0.4, label="Raw")
plt.plot(smooth(all_rewards, 20), label="Smoothed", linewidth=2)
plt.legend()
plt.grid(True)
plt.title("REINFORCE + Baseline on CartPole")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()
